I CANT EXPLAIN HOW MUCH I HATE PREPROCESSING DATA 

In [ ]:
import mne
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
#importing stuff

In [ ]:
file_path = r'file path'
raw = mne.io.read_raw_fif(file_path, preload=True)
#load the data

In [ ]:
raw.filter(1., 40., fir_design='firwin')
#filter

In [ ]:
epoch_length = 1.0  # in seconds
sfreq = raw.info['sfreq']  # sampling frequency
n_samples = int(epoch_length * sfreq)

In [ ]:
data = raw.get_data()

In [ ]:
n_epochs = data.shape[1] // n_samples
X = np.array([data[:, i*n_samples:(i+1)*n_samples] for i in range(n_epochs)])

In [ ]:
y = np.random.randint(0, 2, n_epochs)

In [ ]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [ ]:
print("Data shape:", X.shape)
print("Labels shape:", y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

This is the actual part where i make the model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#import evrythang

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

In [ ]:
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

here comes the training part

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)